<a href="https://colab.research.google.com/github/kentokura/gob/blob/main/gobgob_ipynb_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [301]:
from typing import List
import itertools
import copy

# idは134bitの文字列
# stateの引数からidを作成
def create_id(is_first: bool, is_choise: bool ,choise_board: List[List[bool]],
              choise_hand: List[bool] ,board: List[List[bool]], hand: List[bool]):
  big_list = [int(is_first)] + [int(is_choise)] \
          + list(itertools.chain.from_iterable(choise_board)) + choise_hand \
          + list(itertools.chain.from_iterable(board)) + hand;
  id = "".join(map(str, big_list))
  return id;

# idからstateの引数を作成
def id2is_first(id: str):
  return id[0];
def id2is_choise(id: str):
  return id[1];
def id2choise_board(id: str):
  choise_board = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
  for i, str_bit in enumerate(id[2:2+54]):
      choise_board[i//9][i%9] = int(str_bit)
  return choise_board;
def id2choise_hand(id: str):
  choise_hand = [0,0,0,0,0,0,0,0,0,0,0,0]
  for i, str_bit in enumerate(id[56:56+12]):
      choise_hand[i] = int(str_bit)
  return choise_hand;
def id2board(id: str):
  board = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
           [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
  for i, str_bit in enumerate(id[68:68+54]):
      board[i//9][i%9] = int(str_bit)
  return board;
def id2hand(id: str):
  hand = [0,0,0,0,0,0,0,0,0,0,0,0]
  for i, str_bit in enumerate(id[122:122+12]):
      hand[i] = int(str_bit)
  return hand;

# 正規化
def create_normalization_board(board):
  board = board # 候補
  # 変換表を一つずつためし、cand_boardと比較して小さい方を残す
  convert_tables = [[6,3,0,7,4,1,8,5,2],[8,7,6,5,4,3,2,1,0],[2,5,8,1,4,7,0,3,6],
  [2,1,0,5,4,3,8,7,6],[6,7,8,3,4,5,0,1,2],[0,3,6,1,4,7,2,5,8],[8,5,2,7,4,1,6,3,0]]
  for convert_table in convert_tables:
    cand_board = [
                  [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]];
    for i in range(6):
      for j, num in enumerate(convert_table):
        cand_board[i][j] = board[i][num]
    # cand_pieces と temp_cand_pieces を比較
    if board > cand_board:
      board = cand_board
  return board
def create_normalization_hand(hand):
  hand = hand # 候補
  # "1,0" を "0,1"にするだけ。
  for i in range(6):
    if hand[2*i] == 1 and hand[2*i+1] == 0:
      hand[2*i] = 0
      hand[2*i+1] = 1
  return hand

# 合法手の作成
def create_legal_actions(is_first, is_choise, choise_board, choise_hand, board, hand):
  actions = [
            [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],
            [0,0,0,0,0,0,0,0,0,0,0,0]]
  if is_choise == True: # stateはchoise、ここではchoiseした駒を置く行動を絞り込む
    choise_piece_size = 0 # 0~5
    # 手駒選択時,選択した駒サイズはすべて1にする
    if choise_hand != [0,0,0,0,0,0,0,0,0,0,0,0]:
      for i, piece in enumerate(choise_hand):
        if piece == 1:
          choise_piece_size = i//2
          actions[0][choise_piece_size] = [1,1,1,1,1,1,1,1,1]
        break;
    else: # 盤面, 選択した駒の位置以外をすべて1にする
      for i in range(6):
        if choise_board[i] != [0,0,0,0,0,0,0,0,0]:
          choise_piece_size = i
          actions[0][choise_piece_size] = [1,1,1,1,1,1,1,1,1]
          # 元の位置は再度0にする
          for place, is_exist in enumerate(choise_board[choise_piece_size]):
            if is_exist == 1: 
              actions[0][choise_piece_size][place] = 0
    # choise_piece_size　と同じ大きさ以上の駒が盤面にあるなら、その場所にはおけない
    check_piece_sizes = [[0,3], [0,1,3,4], [0,1,2,3,4,5]]
    for check_piece_size in check_piece_sizes[(choise_piece_size) % 3]:
      for place, is_exist in enumerate(board[check_piece_size]): 
        if is_exist == 1: 
          actions[0][choise_piece_size][place] = 0

  else: # is_choise == False: # stateはput、ここではchoiseする駒を選べる行動を絞り込む
    actions = [copy.deepcopy(board), copy.deepcopy(hand)] # 今ある位置どこでも動かせる前提から始める
    # 選べるのは次のプレイヤーの駒だけ(手番が交代するから)
    if is_first == 1: # 先手の駒を行動不可にする
      for i in [0,1,2]:
        actions[0][i] = [0,0,0,0,0,0,0,0,0]
        for j in range(6):
          actions[1][j] = 0
    else: # 後手の駒を行動不可にする
      for i in [3,4,5]:
        actions[0][i] = [0,0,0,0,0,0,0,0,0]
        for j in range(6):
          actions[1][j+6] = 0
    # ボード上の駒を調べ、LがあるplaceのM,Sを行動不可, MがあるplaceのSを行動不可にする。
    piece_sizes = {"M, S": [1,2,4,5],"S": [2,5]}
    for place in range(9):
      if board[0][place] == 1 or board[3][place] == 1: # L
        for piece_size in piece_sizes["M, S"]:
          actions[0][piece_size][place] = 0
      elif board[1][place] == 1 or board[4][place] == 1: # M
        for piece_size in piece_sizes["S"]:
          actions[0][piece_size][place] = 0
  return actions

# stateクラス
class State():
  def __init__(self, is_first, is_choise, choise_board, choise_hand, board, hand):
    self.is_first = is_first
    self.is_choise = is_choise
    self.choise_board = choise_board
    self.choise_hand = choise_hand
    self.board = board
    self.hand = hand
    self.legal_actions = create_legal_actions(
        is_first,is_choise,choise_board,choise_hand,board, hand)
  # 次の状態の作成
  def next_states(self):
    next_states = []
    next_is_choise = int(not is_choise)
    next_is_first = int(not is_first) if next_is_choise == 1 else is_first # 次がchoiseなら反転
    # 現在is_choise== 1 なら、actionにはchoiseした駒をおける場所が入っている。
    # action[1]にビットが立っていることはない.
    # action[0][type][place]== 1のとき、 board[type][place] = 1にする。
    if is_choise == 1: # 手番を継続, 次はセット, actionをboard, handに反映する
      next_choise_board = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]];
      next_choise_hand = [0,0,0,0,0,0,0,0,0,0,0,0]
      for piece_type_num, piece_type in enumerate(self.legal_actions[0]):
        for place_num, place in enumerate(piece_type):
          print("place_num = {}, place = {}".format(place_num, place))
          if place == 1: # 行動可能ならその箇所を1にする
            print("ififihieghieghihei")
            next_board = copy.deepcopy(self.board)
            next_board[piece_type_num][place_num] = 1
            state = State(next_is_first, next_is_choise,
                    next_choise_board, next_choise_hand,
                    next_board, copy.deepcopy(self.hand))
            next_states.append(state)
    # 現在is_choise== 0 なら、actionにはchoiseできる駒の位置が入っている。
    # action[0][place]== 1のとき、 hand[place] = 0、choise_hand[place] = 1にする。
    # action[0][type][place]== 1のとき、 board[type][place] = 0、choise_board[place] = 1にする。
    else: # is_choise == 0: # 手番を交代、次はチョイス, actionをchise_board, chise_handに反映する
      print("next choise")
      # 盤面からchoiseする
      for piece_type_num, piece_type in enumerate(self.legal_actions[0]):
        for place_num, place in enumerate(piece_type):
          if place == 1: # 行動可能ならその箇所のchoise_boardを1にして、その箇所のboardを0にする
            next_choise_board = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                                 [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]];
            next_board = self.board
            next_choise_board[piece_type_num][place_num] = 1
            next_board[piece_type_num][place_num] = 0
            state = State(next_is_first, next_is_choise,
                    next_choise_board, [0,0,0,0,0,0,0,0,0,0,0,0],
                    next_board, self.hand)
            next_states.append(state)
      # ハンドからchoiseする
      for place_num, place in enumerate(enumerate(self.legal_actions[1])):
        if place == 1: # 行動可能ならその箇所のchoise_handを1にして、その箇所のhandを0にする
          next_choise_hand = [0,0,0,0,0,0,0,0,0,0,0,0]
          next_hand = self.hand
          next_choise_hand[place_num] = 1
          next_hand[place_num] = 0
          state = State(next_is_first, next_is_choise,
                  [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                   [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],
                  next_choise_hand, self.board, next_hand)
          next_states.append(state)    
    return next_states

In [302]:
# テスト
is_first: bool = False
is_choise: bool = False
choise_board: List[List[bool]] =[[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                                 [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
choise_hand: List[bool] = [0,0,0,0,0,0,0,0,0,0,0,0]
board: List[List[bool]] =[[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
hand: List[bool] = [1,1,1,1,1,1,1,1,1,1,1,1]
legal_actions: List[List]= [
  [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],
  [0,0,0,0,0,0,0,0,0,0,0,0]]

is_first: bool = True
is_choise: bool = True
choise_board: List[List[bool]] =[[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                                 [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
choise_hand: List[bool] = [0,0,0,0,0,0,0,0,0,0,0,0]
board: List[List[bool]] =[[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
hand: List[bool] = [0,1,1,1,1,1,1,1,1,1,1,1]
legal_actions: List[List]= [
  [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],
  [1,0,0,0,0,0,0,0,0,0,0,0]]

id = create_id(is_first, is_choise,choise_board,choise_hand,board,hand)
state = State(is_first, is_choise,choise_board,choise_hand,board,hand)
# # idのテスト　
# print("id = {}".format(id))
# print("is_first = {}".format(id2is_first(id)))
# print("is_choise = {}".format(id2is_choise(id)))
# print("choise_board = {}".format(id2choise_board(id)))
# print("choise_hand = {}".format(id2choise_hand(id)))
# print("board = {}".format(id2board(id)))
# print("hand = {}".format(id2hand(id)))

# stateのテスト　
print("state = {}".format(state))
print("is_first = {}".format(state.is_first))
print("is_choise = {}".format(state.is_choise))
print("choise_board = {}".format(state.choise_board))
print("choise_hand = {}".format(state.choise_hand))
print("board = {}".format(state.board))
print("hand = {}".format(state.hand))
print("legal_actions = {}".format(state.legal_actions))
print("next_states = {}".format(state.next_states()))

# 正規化テスト
# hand: List[bool] = [1,0,1,0,1,1,0,1,0,1,0,1]
# print("hand = {}".format(hand))
# print("nrm_hand = {}".format(create_normalization_hand(hand)))

# board = [
#          [1,0,1,0,1,0,1,0,1],[1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
#          [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]];
# print("board = {}".format(board))
# print("nrm_board = {}".format(create_normalization_board(board)))

state = <__main__.State object at 0x7f1e988f3f28>
is_first = True
is_choise = True
choise_board = [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]
choise_hand = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
board = [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]
hand = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
legal_actions = [[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
place_num = 0, place = 0
place_num = 1, place = 0
place_num = 2, place = 0
place_num = 3, place = 0
place_num = 4, place = 0
place_num = 5, place = 0
place_num = 6, place = 0
place_num = 7, place = 0
place_num = 8, pla